# Introduction
State notebook purpose here

### Get source folder and append to sys directory

In [1]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
#pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


### Imports
Import libraries and write settings here.

In [2]:
# Data manipulation
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module

# Visualizations
import matplotlib.pyplot as plt

# Analysis/Modeling
Do work here

In [3]:
import feedparser
feed = "http://dosairnowdata.org/dos/RSS/HoChiMinhCity/HoChiMinhCity-PM2.5.xml"
NewsFeed = feedparser.parse(feed)

In [4]:
from features.calculate_AQI import categorize_AQI
from features.extract_features import data_preprocessing
from data.create_input_for_models import create_input_for_model
train = pd.DataFrame.from_dict(NewsFeed, orient='index')
train
train2 = pd.DataFrame.from_dict(train.loc['entries', :].values[0])
train2 = train2[['title', 'aqi']]
train2.rename(columns={'title': 'time', 'aqi': 'AQI_h'}, inplace=True)
train2 = train2.astype({'time': 'datetime64[ns]', 'AQI_h': 'float'})
train2['site_id'] = 49
train2.set_index(['site_id', 'time'], inplace=True)
train2['AQI_h_label'] = categorize_AQI(train2['AQI_h'])
train2['AQI_h_I'] = train2['AQI_h_label'].cat.codes + 1
train2['Continous length'] = 0
train, y_train = create_input_for_model(train2, timesteps=[5], target_hour=[1], output_path=os.path.join(PROJ_ROOT, "data", "model_input", "aqinow"))

,0
bozo,False
entries,"[{'title': '2021-07-02 16:00:00', 'title_detai..."
feed,"{'title': 'Ho Chi Minh City', 'title_detail': ..."
headers,"{'content-type': 'application/xml', 'content-l..."
etag,"""1d7f-5c633f124a8a2"""
updated,"Sat, 03 Jul 2021 08:34:34 GMT"
updated_parsed,"(2021, 7, 3, 8, 34, 34, 5, 184, 0)"
href,http://dosairnowdata.org/dos/RSS/HoChiMinhCity...
status,200
encoding,utf-8


Feature shape:  (12, 5, 33)
Label shape:  (12,)
Data array:
shape of arr:  (12, 5, 33)
shape of loaded_array:  (12, 5, 33)
Yes, both the arrays are same
Label array:
shape of arr:  (12,)
shape of loaded_array:  (12,)
Yes, both the arrays are same
Input have been created


In [6]:
train2.to_csv("past_data.csv")

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics, initializers
from tensorflow.keras import backend as K
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from data.create_load_transform_processed_data import create_tensorflow_dataset

def get_model_name(timesteps, target_hour):
    return 'model_{}_{}.h5'.format(timesteps, target_hour)

def get_model_weigts_name(timesteps, target_hour):
    return 'weights_{}_{}.ckpt'.format(timesteps, target_hour)
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))
def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)
os.path.abspath(PROJ_ROOT)
model_path = os.path.join(PROJ_ROOT, 'models', 'combined', 'model_5_1')
model = keras.models.load_model(model_path, custom_objects={'LeakyReLU': layers.LeakyReLU(alpha=0.01),
                                               'root_mean_squared_error': root_mean_squared_error})
train_tf, steps_train = create_tensorflow_dataset(train, y_train, 1)

'/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program'

[[[ 1.0000000e+00  1.0000000e+00  0.0000000e+00 ...  5.2000000e+01
    5.2000000e+01  5.2000000e+01]
  [ 1.0000000e+00  1.0000000e+00  0.0000000e+00 ...  5.5000000e+01
    5.5000000e+01  5.5000000e+01]
  [-7.3913044e-01  0.0000000e+00  0.0000000e+00 ...  5.6500000e+01
    5.6500000e+01  5.6500000e+01]
  [-1.3043478e-01  0.0000000e+00  0.0000000e+00 ...  5.3400002e+01
    5.3400002e+01  5.3400002e+01]
  [-4.3478262e-02  0.0000000e+00  0.0000000e+00 ...  5.2500000e+01
    5.2500000e+01  5.2500000e+01]]

 [[ 1.0000000e+00  1.0000000e+00  0.0000000e+00 ...  5.5000000e+01
    5.5000000e+01  5.5000000e+01]
  [-7.3913044e-01  0.0000000e+00  0.0000000e+00 ...  5.6500000e+01
    5.6500000e+01  5.6500000e+01]
  [-1.3043478e-01  0.0000000e+00  0.0000000e+00 ...  5.3400002e+01
    5.3400002e+01  5.3400002e+01]
  [-4.3478262e-02  0.0000000e+00  0.0000000e+00 ...  5.2500000e+01
    5.2500000e+01  5.2500000e+01]
  [-2.1739130e-01  0.0000000e+00  0.0000000e+00 ...  5.2000000e+01
    5.2000000e+01  5.2

In [11]:
train_tf

<BatchDataset shapes: ((1, 5, 33), (1,)), types: (tf.float32, tf.float32)>

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics, initializers
from tensorflow.keras import backend as K
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def get_model_name(timesteps, target_hour):
    return 'model_{}_{}.h5'.format(timesteps, target_hour)

def get_model_weigts_name(timesteps, target_hour):
    return 'weights_{}_{}.ckpt'.format(timesteps, target_hour)
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))
def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

for hour in [1]:
    rmse = []
    r2 = []
    mae = []
    mape = []
    for timestep in range(1, 13):    
        batch_size = 700
        rand = random.randint(0, len(y_test)-40)
        if len(y_test) % batch_size != 0:
            remain_count = len(y_test)%batch_size
            test = test[remain_count:]
            y_test = y_test[remain_count:]
        test_data_tf, test_steps_per_epochs = create_tensorflow_dataset(test, y_test, batch_size)
        
#         model = create_model(batch_size=batch_size, timestep=timestep,features=7, dropout = 0.2)
        model = keras.models.load_model(root_path + 'program/saved_models/latest_model/Hanoi/{}/model_of_{}_hour/{}'.format(
            timestep, hour, get_model_name(timestep, hour)),
                                custom_objects={'LeakyReLU': layers.LeakyReLU(alpha=0.01),
                                               'root_mean_squared_error': root_mean_squared_error})
        model.load_weights('{}{}/model_of_{}_hour/{}'.format(_model_path, timestep, hour,
                                         get_model_weigts_name(timesteps=timestep, target_hour=hour))).expect_partial()
        predict = model.predict(test_data_tf, steps=test_steps_per_epochs).ravel()
        scaler_AQI = MinMaxScaler(feature_range=(-1,1))
        scaler_AQI.fit(thudohanoi_df['AQI_h'].values.reshape(-1, 1))
        y_test = scaler_AQI.inverse_transform(y_test.reshape(-1, 1))
        predict = scaler_AQI.inverse_transform(predict.reshape(-1, 1))
        print("=============================================\n")
#         print("Predict")
#         predict_vs_truth = pd.DataFrame({'predict': predict[rand:rand+20],
#                                         'truth': y_test[rand:rand+20]})
#         print(predict_vs_truth)
        print("R2: {}".format(r2_score(predict, y_test)))
        print("Root mean squared error: {}".format(mean_squared_error(predict, y_test, squared=False)))
        print("Mean absolute percentage error: {}".format(mean_absolute_percentage_error(predict, y_test)))
        print("Mean absolute error: {}".format(mean_absolute_error(predict, y_test)))
        
        rmse.append(mean_squared_error(predict, y_test, squared=False))
        r2.append(r2_score(predict, y_test))
        mape.append(mean_absolute_percentage_error(predict, y_test))
        mae.append(mean_absolute_error(predict, y_test))
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot()
    ax.plot(range(1,13),rmse)
    ax.plot(range(1,13),r2)
    ax.plot(range(1,13),mape)
    ax.plot(range(1,13),mae)

array([ 0.23076923, -0.30769231,  0.69230769, -0.15384615, -0.38461538,
       -0.30769231, -0.46153846, -1.        , -0.53846154, -0.76923077,
       -0.23076923, -0.46153846])

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here